In [5]:
%pip install numpy
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 10.8 MB 573 kB/s eta 0:00:01
     |████████████████████████████████| 509 kB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 348 kB 24.7 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### Imports and Dataset Loading

In [6]:
import numpy as np 
import pandas as pd

In [15]:
df = pd.read_csv('dataset/dataset_csv/JM1.csv')

print(df.head())

   LOC_BLANK  BRANCH_COUNT  LOC_CODE_AND_COMMENT  LOC_COMMENTS  \
0        3.0           1.0                   0.0           0.0   
1        3.0           3.0                   0.0           0.0   
2        1.0           5.0                   0.0           0.0   
3        5.0           9.0                   0.0           4.0   
4        4.0           1.0                   0.0           0.0   

   CYCLOMATIC_COMPLEXITY  DESIGN_COMPLEXITY  ESSENTIAL_COMPLEXITY  \
0                    1.0                1.0                   1.0   
1                    2.0                1.0                   1.0   
2                    3.0                1.0                   1.0   
3                    5.0                5.0                   1.0   
4                    1.0                1.0                   1.0   

   LOC_EXECUTABLE  HALSTEAD_CONTENT  HALSTEAD_DIFFICULTY  HALSTEAD_EFFORT  \
0            12.0             57.51                 4.26          1046.00   
1             9.0             20.0